<a href="https://colab.research.google.com/github/olalepek/Text-Mining---Agriculture/blob/main/%5BPreprocessing%5D_Agriculture_Text_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rispy pandas
!pip install spacy
import rispy
import numpy as np
import pandas as pd
import re
import nltk
import spacy
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')


In [ ]:
ris_file_path ='/content/drive/MyDrive/Studia DSE/combined_file.ris'

with open(ris_file_path, 'r') as bibliography_file:
    entries = rispy.load(bibliography_file)

df = pd.DataFrame(entries)
df.iloc[1]

Selecting only specific columns from dataset, dropping columns with NAN values, transforming the cell values into strings for easier analysis

In [ ]:
selected = ['title','alternate_title1','year','authors','abstract','keywords','secondary_title','notes','type_of_work']
dataset = df[selected]
dataset = dataset.dropna(axis=0, how="any")
dataset = dataset.astype(str)

Porter Stemmer

In [ ]:
processed=[]

for abstract in dataset['abstract']:

    abstract = re.sub('[^a-zA-Z]', ' ', abstract) #replacing any punctuation or anything that is not ^ a-z and A-Zletter with the space
    abstract = abstract.lower() #lowercase all the words
    abstract = abstract.split()# splitting the abstract into words

      # Stemming the words to keep only the roots using Porter Stemmer
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    abstract = [ps.stem(word) for word in abstract if not word in set(all_stopwords)]
    abstract = ' '.join(abstract)
    processed.append(abstract)

dataset['processed_porter_stemmer'] = processed

Lemmitizing the text

In [ ]:
nlp = spacy.load("en_core_web_sm")
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

dataset['lemmatized_text'] = dataset['abstract'].apply(lemmatize_text)

In [ ]:
processed2=[]

for abstract in dataset['lemmatized_text']:
  abstract = re.sub('[^a-zA-Z]', ' ', abstract) #replacing any punctuation or anything that is not ^ a-z and A-Zletter with the space
  abstract = abstract.lower() #lowercase all the words
  all_stopwords = stopwords.words('english')
  words = word_tokenize(abstract)
  filtered_abstract = " ".join(word for word in words if word not in all_stopwords)
  processed2.append(filtered_abstract)
dataset['processed_lemmatized'] = processed2


Comparing the Porter Stemmer with Lemma

In [ ]:
selected = ['abstract','processed_porter_stemmer','processed_lemmatized' ]
dataset_view = dataset[selected]

dataset_view.head()

,abstract,processed_porter_stemmer,processed_lemmatized
1,Effects of soil pH on weak acid and weak base ...,effect soil ph weak acid weak base herbicid ad...,effect soil ph weak acid weak base herbicide a...
2,Growth responses of mesquite (Prosopis juliflo...,growth respons mesquit prosopi juliflora sw dc...,growth response mesquite prosopis juliflora sw...
3,Field experiments with rice-wheat rotation wer...,field experi rice wheat rotat conduct five con...,field experiment rice wheat rotation conduct f...
4,Application of higher levels (60 and 90 kg N h...,applic higher level kg n ha nitrogen fertil ur...,application high level kg n ha nitrogen fertil...
5,A study relating N uptake by wheat to several ...,studi relat n uptak wheat sever n soil test co...,study relate n uptake wheat several n soil tes...


**Findings**

The Porter Stemmer reduces the word to its root, however this root is not always a good representation of the meaning. For the purpose of the further analysis the lemmitized and processed text will be used.